In [1]:
# import libraries
import numpy as np
import os
import sklearn
import sklearn.neighbors
import matplotlib.pyplot as plt
from IPython import display
%matplotlib inline
import itertools
import time
import random
import load_data

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [3]:
data_dir = './input'# define here the directory where you have your data

In [4]:
# labels for the three classes in this classification problem.
# These are the same names used in folders for raw training data
cervixTypes = ["Type_1", "Type_2", "Type_3"]
n_classes = len(cervixTypes)
print('Classification problem with {} classes'.format(n_classes))

Classification problem with 3 classes


In [5]:
data, labels = load_data.load_train()

load from hdf5 file

 Doooone :)


In [6]:
samples_per_class = [249, 781, 450]

In [7]:
def split_training_validation_datasets(x, y, samples_per_class, val_percentage=0.3, val_balanced=True):
    """
    Derive a training and a validation datasets from a given dataset with
    data (x) and labels (y). By default, the validation set is 30% of the
    training set, and it has balanced samples across classes. When balancing,
    it takes the 30% of the class with less samples as reference.
    """
    n_cv_samples_per_class = int(np.amin(samples_per_class) * 0.3)
    
    mask = np.zeros(x.shape[0], dtype=bool)
    low = 0
    high = 0
     
    for c in samples_per_class:
        high = high + c
        ids_cv = np.random.choice(np.arange(low,high),n_cv_samples_per_class, replace = False)
        mask[ids_cv] = True
        low = low + c   
    
    x_train = x[~mask,:, :, :]
    y_train = y[~mask, :]

    x_validation = x[mask,:, :, :]
    y_validation = y[mask, :]
    
    return x_train, y_train, x_validation, y_validation



In [8]:
x_train, y_train, x_validation, y_validation = split_training_validation_datasets(data, labels, samples_per_class)

In [15]:
batch_size = 32
num_classes = 3
epochs = 12

In [16]:
img_rows, img_cols = 400, 400

In [11]:
# if K.image_data_format() == 'channels_first':
#     x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
#     x_validation = x_validation.reshape(x_validation.shape[0], 1, img_rows, img_cols)
#     input_shape = (1, img_rows, img_cols)
# else:
#     x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
#     x_validation = x_validation.reshape(x_validation.shape[0], img_rows, img_cols, 1)
#     input_shape = (img_rows, img_cols, 1)

ValueError: cannot reshape array of size 603840000 into shape (1258,400,400,1)

In [17]:
input_shape = (img_rows, img_cols, 3)

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_validation, y_validation))
model.save_weights('simple_model.h5')
# score = model.evaluate(x_validation, y_validation, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

Train on 1258 samples, validate on 222 samples
Epoch 1/12
1152/1258 [==========================>...] - ETA: 696s - loss: 6.9036 - acc: 0.5521